# Library

In [9]:
import os
from dotenv import load_dotenv 
import yfinance as yf
from datetime import datetime
from gtts import gTTS
from langchain.prompts import PromptTemplate
from langchain.agents import Tool, create_react_agent 
from langchain.tools import DuckDuckGoSearchResults
from langchain_openai import OpenAI
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.schema import AgentFinish
import re

# 環境変数の読み込み
load_dotenv()  # .envファイルから環境変数を読み込み
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# GPTモデルの設定
model_name = 'gpt-4o-mini-2024-07-18'  # 'gpt-4o-2024-08-06'

# OpenAI APIのキーをセット
llm = OpenAI(temperature=0.5, api_key=os.getenv('OPENAI_API_KEY'))

# Function

In [10]:
# DuckDuckGo検索ツールの作成
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchResults(),
    description="Use this tool to search for information on the web."
)

# 独自Python REPLツールの作成
def execute_python_code(code: str):
    try:
        # 入力されたコードを実行
        exec_globals = {}
        exec(code, exec_globals)
        return exec_globals.get("result", "No result variable found.")
    except Exception as e:
        return f"Error: {str(e)}"

python_tool = Tool(
    name="Python Executor",
    func=execute_python_code,
    description="Execute Python code."
)

# 株価取得ツールの作成
def get_stock_price(ticker: str, start_date: str, end_date: str):
    stock = yf.Ticker(ticker)
    # 指定された期間の株価を取得
    stock_info = stock.history(start=start_date, end=end_date)
    
    if not stock_info.empty:
        # 期間内の終値を取得
        closing_price = stock_info['Close'][-1]
        return f"The stock price of {ticker} on {end_date} is {closing_price}."
    else:
        return f"Could not retrieve stock price for {ticker} from {start_date} to {end_date}."

# ツールの定義
stock_tool = Tool(
    name="Stock Price Checker",
    func=get_stock_price,
    description="Retrieve the stock price for a specific company (ticker symbol) for a given time period."
)

# 自動音声合成ツールの作成
def text_to_speech(text: str, language: str = 'en'):
    tts = gTTS(text=text, lang=language)
    tts.save("output.mp3")
    os.system("mpg321 output.mp3")  # オーディオを再生する
    return "The audio has been generated and played."

speech_tool = Tool(
    name="Text to Speech",
    func=text_to_speech,
    description="Convert text to speech and play the audio."
)

# Custom Output Parser

In [11]:
# カスタム出力パーサーの作成
class CustomOutputParser(ReActSingleInputOutputParser):
    def parse(self, text: str):
        # "Final Answer:" があれば、その部分を最終結果とみなす
        if "Final Answer:" in text:
            final_answer = text.split("Final Answer:")[-1].strip()
            return AgentFinish({"output": final_answer}, text)
        
        # 通常の最終出力を想定し、"Action"や"Final Answer"がない場合でも出力をそのまま返す
        return AgentFinish({"output": text.strip()}, text)

# Prompt Template

In [12]:
# ReActエージェント用のプロンプトテンプレートの作成
react_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template=(
        "You are a knowledgeable AI with access to the following tools:\n"
        "{tool_names}\n\n"
        "You can use these tools to answer the following question:\n"
        "{input}\n\n"
        "{agent_scratchpad}"
    )
)

# ReAct base Agent

In [13]:
# ReActベースのエージェントを初期化
tools = [search_tool, python_tool, stock_tool, speech_tool]
tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
tool_names = ", ".join([tool.name for tool in tools])

# カスタム出力パーサーを使用してエージェントを作成
output_parser = CustomOutputParser()
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt.partial(
        tools=tool_descriptions,
        tool_names=tool_names
    ),
    output_parser=output_parser
)

# Question

In [17]:
question = "What is the latest stock price of AAPL and do a Python calculation of result = 5 + 5?"

# `intermediate_steps` を初期化してエージェントを呼び出す
response = agent.invoke({
    "input": question,
    "agent_scratchpad": "",
    "intermediate_steps": []
})

# AgentFinish オブジェクトの return_values から結果を取得
formatted_output = f"""
Result:
-----------------------------------------
{response.return_values['output']}

Log:
-----------------------------------------
{response.log if hasattr(response, 'log') else 'No log information available.'}
"""

print(formatted_output)


Result:
-----------------------------------------
The latest stock price of AAPL is $127.79 per share. Using the Python Executor, the result of 5 + 5 is 10.

Log:
-----------------------------------------
The latest stock price of AAPL is $127.79 per share. Using the Python Executor, the result of 5 + 5 is 10.



In [19]:
question = "日本で一番高い山の標高は何メートル？その高さの平方根は？"

# `intermediate_steps` を初期化してエージェントを呼び出す
response = agent.invoke({
    "input": question,
    "agent_scratchpad": "",
    "intermediate_steps": []
})

# AgentFinish オブジェクトの return_values から結果を取得
formatted_output = f"""
Result:
-----------------------------------------
{response.return_values['output']}

Log:
-----------------------------------------
{response.log if hasattr(response, 'log') else 'No log information available.'}
"""

print(formatted_output)


Result:
-----------------------------------------
日本で一番高い山の標高は、富士山の3776メートルです。その高さの平方根は61.46メートルです。

Log:
-----------------------------------------
日本で一番高い山の標高は、富士山の3776メートルです。その高さの平方根は61.46メートルです。

